In [14]:
## Chaning path and importing modules
!pip3 install -r requirements.txt

import os
import requests 
from sys import platform
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import json

In [21]:
## urlwrite function.

def urlwrite(url, fileName):
    response = requests.get(url)

    for i in range(5):
        if response.status_code == 200:
            break
        response = requests.get(url)
    else:
        raise Exception("Internet not working")
    
    with open(fileName, 'w') as fptr:
        fptr.write(response.text)


In [22]:
## Required constants

dummyFileName = 'data/dummy.csv'
DateFormat = '%Y-%m-%d'
fileName = 'cityAndStateNames.json'

In [23]:
## Get district names and get state names
def getDistrictNames():
    with open(fileName) as fptr:
        return json.loads(fptr.read())['cityNames']

def getStateNames():
    with open(fileName) as fptr:
        return json.loads(fptr.read())['stateNames']

In [24]:
ifplotPR = True
dists = True
debug = False
ifprint = True
span = 7

In [25]:
fullName, sb_number,  name = None, None, None 

In [26]:
if dists:
    fullName = 'https://api.covid19india.org/csv/latest/districts.csv'
    name = getDistrictNames()
    tt1 = pd.to_datetime('2020-4-26')
    sb_number = 4
else:
    fullName = 'https://api.covid19india.org/csv/latest/states.csv'
    name = getStateNames()
    tt1 = pd.to_datetime('2020-3-14')
    sb_number = 5

In [27]:
urlwrite(fullName, dummyFileName)

tableAll = pd.read_csv(dummyFileName, na_values={'Tested':['']})
tableAll.Date = pd.to_datetime(tableAll.Date, format="%Y-%m-%d")

span1 = 14

mean_si, min_mean_si, max_mean_si = 4.7, 3.7, 6.0
std_si, min_std_si, max_std_si = 2.9, 1.9, 4.9 

In [28]:
for n in range(len(name)):

    Location = name[n]

    # To avoid scope related problems.
    tableLocation = None
    R, D, C, O, T = None, None, None, None, None

    if dists:
        tableLocation = tableAll[(tableAll.District == Location) & (tableAll.Date >= pd.to_datetime('1-3-2020', format='%d-%m-%Y'))]

        R = tableLocation['Recovered'].to_numpy()
        D = tableLocation['Deceased'].to_numpy()
        C = tableLocation['Confirmed'].to_numpy()
        O = tableLocation['Other'].to_numpy()
        T = tableLocation['Tested'].to_numpy()
    else:
        tableLocation = tableAll[(tableAll.State == Location) & (tableAll.Date >= pd.to_datetime('1-3-2020', format='%d-%m-%Y'))]

        R = tableLocation['Recovered'].to_numpy()
        D = tableLocation['Deceased'].to_numpy()
        C = tableLocation['Confirmed'].to_numpy()
        O = tableLocation['Other'].to_numpy()
        T = tableLocation['Tested'].to_numpy()
 
        A = C - D - O - R # Active cases

        dailyC = np.diff(C)

        if debug:
            fig = plt.figure()

            plt.plot(date, C)
            xlabel('Date')
            ylabel('Confirmed Cases')
            plt.legend()

            plt.show()